In [10]:
import autogen
from dotenv import load_dotenv
import os
load_dotenv()

True

In [11]:
config_list = [
    {
        "model": "gpt-4-1106-preview",
        "api_key": os.getenv("OPENAI_API_KEY"),
    },
    {
        "model": "gpt-4",
        "api_key": os.getenv("OPENAI_API_KEY")
    },
]

llm_config={
    # "cache_seed":42,
    "temperature":0,
    "config_list":config_list
}

In [6]:
planner = autogen.AssistantAgent(
    name="Planner",
    llm_config=llm_config,
    system_message="You are a helpful AI assistant. You suggest coding and reasoning steps for another AI assistant. Do not suggest concrete code. For any action beyond writing code or reasoning, convert it to a step that can be implemented by writing code. For example, browsing the web can be implemented by writing code that reads and prints the content of a web page. Finally, inspect the execution result. If the plan is not good, suggest a better plan. If the execution is wrong, analyze the error and suggest a fix."
)

planner_user = autogen.UserProxyAgent(
    name="planner_user",
    max_consecutive_auto_reply=0,
    human_input_mode="NEVER"
)

In [12]:
def ask_planner(message):
    planner_user.initiate_chat(planner, message=message)
    return planner_user.last_message()["content"]

In [14]:
assistant = autogen.AssistantAgent(
    name = "assistant",
    llm_config = {
        "temperature": 0,
        "timeout": 600,
        "config_list": config_list,
        "functions": [
            {
                "name": "ask_planner",
                "description": "ask planner to 1. get a plan for finishing a task, 2. verify the execution result of the plan and potentially suggest new plan.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "question to ask_planner. Make sure the question includes enough context, such as the code and the execution result."
                        }
                    },
                    "required": ["message"]
                }
            }
        ]
    }
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "planning"},
    function_map={"ask_planner": ask_planner},
)

In [ ]:
# user_proxy.initiate_chat(
#     assistant,
#     message = "Suggest a fix to an open good first issue of flaml"
# )